# A Tutorial for Package TFCox(DeepHit)

## About the Package
`TFCox` is a python package for predicting hazard rate with deep learning method in survival 
analysis. It is developed based on `Tensorflow==1.13.1`. Due to the advent of `Tensorflow2.x`,
which deprecates and reintegrates so many modules and APIs in `Tensorflow1.x` so that greater 
user experience can be achieved, this package is the only version developed by `Tensorflow1.x`. The future version will be developed based on `Tensorflow2.x`. This notebook will show you how to use deep learning in survival analysis.

## Installation
Please follow the instructions on [README](../README.md) to install `TFCox` package.

## Reference
This package implements the model `DeepHit` in the paper of Changhee Lee et.al. Please refer to 

[1] Changhee Lee, William R Zame, Jinsung Yoon, and Mihaela van der Schaar. Deephit: A deep learning approach to survival analysis with competing risks. In Thirty-Second AAAI Conference on Artificial Intelligence, 2018.http://medianetlab.ee.ucla.edu/papers/AAAI_2018_DeepHit

## A Simulated Example for DeepHit Using Synthetic Dataset

### Load necessary modules

In [1]:
import numpy as np
import tensorflow as tf
from TFCox.models import deephit
from TFCox.datasets import from_deephit
from TFCox.evals.concordance import c_index_deephit
from TFCox.evals.brier_score import brier_score
from sklearn.model_selection import train_test_split

### Load dataset

In [2]:
x_dim, (data, time, label), (mask1, mask2) = from_deephit.load_synthetic(norm_mode='standard')
eval_times = [12, 24, 36]
_, num_event, num_category = np.shape(mask1)

In [4]:
np.unique(label)

array([0, 1, 2], dtype=int64)

The data strcture of Synthetic is shown below. Columns 0 to 11 are covariates. 
Column 12 is survivial time(including censored times), and column 13 is the competing risks.
There are two competing risks, and 0 indicates right censoring.

In [31]:
import pandas as pd
pd.DataFrame(np.hstack((data, time, label))).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.443723,-0.034609,-0.019945,-0.036814,-0.607319,-0.164873,0.523187,-0.252821,-0.299733,0.273318,0.624771,0.237494,0.0,0.0
1,0.011761,-0.840963,0.522711,0.702957,0.155245,-0.123634,-1.470146,-0.288210,-0.349810,-0.314373,-0.495444,-1.071698,1.0,0.0
2,0.442109,1.631825,-1.839474,0.340529,-1.192893,0.355144,0.231882,-0.693089,-1.748800,1.148612,-0.633124,1.743149,34.0,2.0
3,0.624841,-0.611956,-0.335367,-0.981379,0.409087,-0.632022,2.140804,0.110192,-1.174291,-2.841505,-0.363183,-0.865092,9.0,0.0
4,1.244371,-0.184288,-0.187585,-1.069148,-0.061790,-0.155677,-1.324641,-0.694930,-0.139073,0.879568,1.006352,0.748075,2.0,0.0


### Split the data into training set and test set

In [32]:
(tr_data, te_data, tr_time, te_time, tr_label, te_label,
 tr_mask1, te_mask1, tr_mask2, te_mask2) = train_test_split(data, time, label, mask1, mask2,
                test_size=0.20, random_state=0)

### Network settings 

In [33]:
input_dims = {
    'x_dim':x_dim,
    'num_event': num_event,
    'num_category':num_category
}
settings = {
    'h_dim_shared':10,
    'h_dim_CS':5,
    'num_layers_shared':10,
    'num_layers_CS':5,
    'activation':tf.nn.relu,
    'initial_weight': tf.contrib.layers.xavier_initializer()
}

The loss function includes two parts: the negative log partial likelihood and ranking loss.
$$
\operatorname { loss } _ { L } = - \sum _ { i = 1 } ^ { N } \left[ D _ { i } \log \left( y _ { e _ { i } } \left( \mathbf { x } _ { i } \right) \right) + \left( 1 - D _ { i } \right) \log \left( \hat { S } \left[ T _ { i } | x _ { i } \right] \right) \right]
$$

$$
\operatorname { loss } _ { \mathrm { rank } } = \sum _ { i , j } D _ { i } \mathbb { 1 } \left\{ T _ { i } < T _ { j } \right\} \exp \left( \frac { \hat { S } \left( T _ { i } | \mathbf { x } _ { i } \right) - \hat { S } \left( T _ { i } | \mathbf { x } _ { j } \right) } { \sigma } \right)
$$

$$
\text{loss} = \alpha \text{loss}_{L} + (1-\alpha) \text{loss}_{\text{rank}}.
$$

### Start a session

In [35]:
tf.reset_default_graph()
config = tf.ConfigProto()
sess = tf.Session(config=config)
model = deephit.DeepHit(sess, 'DeepHit', input_dims, settings)
sess.run(tf.global_variables_initializer())

### Train the neural network and get corresponding metric
Metrics inludes cause-specific concordance index and Brier score
$$
C ^ { \mathrm { td } } = \mathrm { P } \left\{ \hat { S } \left( T _ { i } | \mathbf { x } _ { i } \right) < \hat { S } \left( T _ { i } | \mathbf { x } _ { j } \right) | T _ { i } < T _ { j } , D _ { i } = 1 \right\}
$$

$$
\mathrm { BS } ( t ) = \frac { 1 } { N } \sum _ { i = 1 } ^ { N } \left[ \frac { \hat { S } \left( t | \mathbf { x } _ { i } \right) ^ { 2 } \mathbb { 1 } \left\{ T _ { i } \leq t , D _ { i } = 1 \right\} } { \hat { G } \left( T _ { i } \right) } + \frac { \left( 1 - \hat { S } \left( t | \mathbf { x } _ { i } \right) \right) ^ { 2 } \mathbb { 1 } \left\{ T _ { i } > t \right\} } { \hat { G } ( t ) } \right].
$$

In [36]:
pred = model.predict(te_data)

c_index = np.zeros([num_event, len(eval_times)])
brier_socre = np.zeros([num_event, len(eval_times)])

for t, time in enumerate(eval_times):
    eval_horizon = int(time)
    if eval_horizon >= num_category:
        print('ERROR: evaluation horizon is out of range')
        c_index[:, t]  = -1
        brier_socre[:, t] = -1
    else:
        risk = np.sum(pred[:, :, :(eval_horizon + 1)], axis=2)
        for k in range(num_event):
            c_index[k, t] = c_index_deephit(risk[:, k], 
                                            te_time,(te_label[:, 0] == k + 1).astype(int), 
                                            eval_horizon)
            brier_socre[k, t] = brier_score(risk[:, k], 
                                        te_time,(te_label[:, 0] == k + 1).astype(int),
                                        eval_horizon)

In [10]:
c_index

array([[0.48903082, 0.49261316, 0.50063936],
       [0.50526229, 0.51791122, 0.52382519]])

In [13]:
brier_socre

array([[0.15170425, 0.17885536, 0.18831516],
       [0.1465261 , 0.17281726, 0.18209337]])